In [1]:
!git clone https://github.com/ellenguyen/CIS4496_EY.git

fatal: destination path 'CIS4496_EY' already exists and is not an empty directory.


In [2]:
import os
os.chdir('/kaggle/working/CIS4496_EY/given/AugmentedData')

In [3]:
from sklearn.model_selection import train_test_split
import shutil

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms
import os
from PIL import Image
import numpy as np

In [5]:
class CustomDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = sorted(os.listdir(os.path.join(root, "images")))
        self.labels = sorted(os.listdir(os.path.join(root, "annotations")))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        label_path = os.path.join(self.root, "annotations", self.labels[idx])
        img = Image.open(img_path).convert("RGB")
        with open(label_path) as f:
            boxes = []
            labels = []
            for line in f:
                annot = line.strip().split()
                label = int(annot[0])
                bbox = [float(n) for n in annot[1:]]
                # Convert to [xmin, ymin, xmax, ymax]
                boxes.append([bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]])
                labels.append(label)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [7]:
# Data transformations
data_transforms = transforms.Compose([
    transforms.ToTensor(),
])

dataset_train = CustomDataset('/kaggle/working/CIS4496_EY/given/AugmentedData/train', transforms=data_transforms)
dataset_test = CustomDataset('/kaggle/working/CIS4496_EY/given/AugmentedData/test', transforms=data_transforms)

# data_loader_train = DataLoader(dataset_train, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
data_loader_test = DataLoader(dataset_test, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)), num_workers=2)

In [9]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

def get_model_instance_segmentation(num_classes):
    # Load an instance segmentation model pre-trained on COCO
    weights = FasterRCNN_ResNet50_FPN_Weights.COCO_V1
    model = fasterrcnn_resnet50_fpn(weights=weights)

    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

num_classes = 4  # Your number of classes here, plus one for the background
model = get_model_instance_segmentation(num_classes)

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [13]:
from torch.optim import SGD
from torchvision.ops import batched_nms


# Use SGD optimizer
optimizer = SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in data_loader_train:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {losses.item()}')

Epoch 1, Loss: 0.16892239451408386
Epoch 2, Loss: 0.3106198012828827
Epoch 3, Loss: 0.16584014892578125
Epoch 4, Loss: 0.30579984188079834
Epoch 5, Loss: 0.26340627670288086
Epoch 6, Loss: 0.1797814816236496
Epoch 7, Loss: 0.10230064392089844
Epoch 8, Loss: 0.09621697664260864
Epoch 9, Loss: 0.2646562159061432
Epoch 10, Loss: 0.194094717502594


In [15]:
from torchvision.ops import box_iou

def evaluate(model, data_loader, device):
    model.eval()
    detection_threshold = 0.5
    all_precisions = []

    with torch.no_grad():
        for images, targets in data_loader:
            images = list(img.to(device) for img in images)
            outputs = model(images)

            for i, image in enumerate(images):
                output = outputs[i]
                target = targets[i]

                output_boxes = output['boxes'].cpu()
                target_boxes = target['boxes'].cpu()

                iou = box_iou(output_boxes, target_boxes)
                
                # Consider detections with IoU > detection_threshold as True Positives (TP)
                max_iou, _ = torch.max(iou, dim=1)
                tp = max_iou > detection_threshold
                num_tp = tp.sum().item()
                
                # False Positives (FP) are detections not matching any ground truth box
                num_fp = (max_iou <= detection_threshold).sum().item()
                
                # Precision for this image
                if num_tp + num_fp > 0:
                    precision = num_tp / (num_tp + num_fp)
                else:
                    precision = 0
                
                all_precisions.append(precision)

    # Calculate the mean precision across all test images
    mean_precision = sum(all_precisions) / len(all_precisions)

    print(f"Mean Precision: {mean_precision}")

    return mean_precision

# Call the evaluate function
evaluate(model, data_loader_test, device)

Mean Precision: 0.011192731992571472


0.011192731992571472